# EGEDA Dashboard

In [ ]:
import numpy as np
import pandas as pd
import param
import panel as pn
pn.extension()
import hvplot.pandas

In [ ]:
df = pd.read_csv('../data/EGEDA_2019_09_25_tidy.csv')

In [ ]:
economies = [
    '01_AUS',
    '02_BD',
    '03_CDA',
    '04_CHL',
    '05_PRC',
    '06_HKC',
    '07_INA',
    '08_JPN',
    '09_ROK',
    '10_MAS',
    '11_MEX',
    '12_NZ',
    '13_PNG',
    '14_PE',
    '15_RP',
    '16_RUS',
    '17_SIN',
    '18_CT',
    '19_THA',
    '20_USA',
    '21_VN']

In [ ]:
df = df[(df['Year']>=1990) & (df['Year']<=2016) & (df['Units']=='PJ')]
df = df.loc[df['Economy'].isin(economies)]
df = df.reset_index(drop=True)
df = df.rename({'Value':'Energy'},axis=1)

In [ ]:
df.tail(2)

In [ ]:
df_co2 = pd.read_csv('../data/CO2 emissions prepared.csv')

In [ ]:
df_co2 = df_co2[df_co2['Item']=='Total CO2 Emission']
df_co2 = df_co2.reset_index(drop=True)
df_co2 = df_co2.rename({'Value':'Emissions'},axis=1)

In [ ]:
df_co2.head(2)

Create a dashboard:

In [ ]:
item_codes = df['Item_Code'].unique().tolist()

In [ ]:
class EGEDA(param.Parameterized):
    economy_selector = param.ObjectSelector(default="05_PRC", objects=economies)
    item_selector = param.ObjectSelector(default="12 Total final energy consumption", objects=item_codes)
    year_range = param.Range(default=(1990, 2016), bounds=(1990, 2016))
    show_legend = param.Boolean(True, doc="Show the legend")
    
    ylabel = 'PJ'
   
    def get_energy_data(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        economy = self.economy_selector
        item = self.item_selector
        mask = (df['Economy'] == economy) & (df['Year'] > start_date) & (df['Year'] <= end_date) & (df['Item_Code']==item)
        
        _df = df.loc[mask]
        
        df_pivot = _df.drop(['Economy','Units','Item_Code'],axis=1).pivot_table(index=['Year'],columns='Fuel',values='Energy')
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
    
    def get_emissions_data(self):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        economy = self.economy_selector
        mask = (df_co2['Economy'] == economy) & (df_co2['Year'] > start_date) & (df_co2['Year'] <= end_date)  
        
        _df = df_co2.loc[mask]
        
        df_pivot = _df.drop(['Economy'],axis=1).pivot_table(index=['Year'],columns='Item',values='Emissions')
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
    
    def get_ee_data(self):
        energy_data  = self.get_energy_data()
        energy_data = energy_data.set_index('Year')
        
        emissions_data  = self.get_emissions_data()
        emissions_data = emissions_data.set_index('Year')
        
        df_merged = energy_data.join(emissions_data)
        df_merged = df_merged.reset_index()
        
        return df_merged
    
    def energy_area(self):
        data  = self.get_energy_data()
        title = self.item_selector + ' in PJ' + ' for ' + self.economy_selector
        
        plot = data.hvplot.area(x='Year', 
                               y=['Coal','Elec','Gas','PetP','TotRen'],
                               stacked=True, 
                               alpha=0.4,
                               width=600,
                               legend=self.show_legend,
                               hover_cols='all',
                               title=title)
        return plot
    
    def emissions_plot(self):
        data  = self.get_emissions_data()
        title = 'Total CO2 emissions for ' + self.economy_selector
        
        plot = data.hvplot(x='Year', 
                           y=['Total CO2 Emission'],
                           value_label='million tons',
                           title=title,
                           alpha=0.4,
                           width=600)
        return plot

    
    def ee_plot(self):
        data  = self.get_ee_data()
        title = 'Total CO2 emissions vs Total Energy for ' + self.economy_selector
        
        plot = data.hvplot.scatter(x='Total CO2 Emission', 
                           y='Tot',
                           c='Year',
                           value_label='million tons',
                           title=title,
                           width=600)
        return plot
    
    
    def hv_table(self):
        data  = self.get_energy_data()
        
        table = data.hvplot.table(['Year', 'Coal','Elec','Gas','PetP','TotRen'], width=420)
        return table
    
    def emissions_table(self):
        data  = self.get_emissions_data()
        
        table = data.hvplot.table(['Year', 'Total CO2 Emission'], width=420)
        return table
    
    def ee_table(self):
        data  = self.get_ee_data()
        
        table = data.hvplot.table(['Year', 'Tot','Total CO2 Emission'], width=420)
        return table
    
e = EGEDA(name='') 

In [ ]:
dashboard = pn.Column(e.param,
                      pn.Row(e.energy_area,e.hv_table),
                      pn.Row(e.emissions_plot,e.emissions_table),
                      pn.Row(e.ee_plot,e.ee_table)
                     )

In [ ]:
dashboard